In [1]:
!pip install catboost
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.9 MB/s eta 0:00:00


Подбор гиперпараметров catboost

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
import csv
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer

with open('public_data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

df = pd.read_csv('public_data.csv')
mean = df["Arrival Delay in Minutes"].mean()

for i in range(len(data)):
  if '' in data[i]:
    data[i][data[i].index('')] =  mean

with open('public_data.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)
df = pd.read_csv('public_data.csv',index_col='Row_id')
df=df.drop(columns=["id"])
y = df['satisfaction']

cat_columns = ['Gender','Customer Type','Type of Travel','Class']
real_columns = ['Age','Flight Distance','Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment',
                'On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness','Departure Delay in Minutes','Arrival Delay in Minutes']
cat_df = df[cat_columns]
real_df = df[real_columns]
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_data = ohe.fit_transform(cat_df)
one_hot_data = pd.DataFrame(one_hot_data,index=cat_df.index)
y[y=='neutral or dissatisfied']=0
y[y=='satisfied']=1
data_norm = (real_df - real_df.min())/ (real_df.max() - real_df.min())
data_norm =  pd.DataFrame(data_norm,columns=data_norm.columns,dtype=float)
one_hot_data = pd.DataFrame(one_hot_data,columns=one_hot_data.columns,dtype=float)
y= pd.DataFrame(y,dtype=float)
data_norm.columns = data_norm.columns.astype(str)
one_hot_data.columns = one_hot_data.columns.astype(str)
train_real_data, test_real_data, \
train_one_hot_data, test_one_hot_data, \
y_train, y_test = train_test_split(data_norm, one_hot_data,
                                   y, train_size=0.8, random_state=85)
X_train = pd.concat([train_real_data, train_one_hot_data], axis='columns')
X_test= pd.concat([test_real_data, test_one_hot_data], axis='columns')

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Функция для оптимизации гиперпараметров
def objective(params):
    clf = CatBoostClassifier(
        iterations=int(params['iterations']),
        depth=int(params['depth']),
        loss_function='Logloss',
        task_type="CPU"
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return {'loss': -acc, 'status': STATUS_OK}

# Пространство поиска гиперпараметров
space = {
    'iterations': hp.quniform('iterations', 100, 1000, 50),
    'depth': hp.quniform('depth', 2, 16, 1)
}

# Оптимизация гиперпараметров
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

# Вывод лучших найденных гиперпараметров
print('Best hyperparameters:')
print('iterations:', int(best['iterations']))
print('depth:', int(best['depth']))

Подбор весов в ансамбле

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
import csv
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer

with open('public_data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

df = pd.read_csv('public_data.csv')
mean = df["Arrival Delay in Minutes"].mean()

for i in range(len(data)):
  if '' in data[i]:
    data[i][data[i].index('')] =  mean

with open('public_data.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)
df = pd.read_csv('public_data.csv',index_col='Row_id')
df=df.drop(columns=["id"])
y = df['satisfaction']

cat_columns = ['Gender','Customer Type','Type of Travel','Class']
real_columns = ['Age','Flight Distance','Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment',
                'On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness','Departure Delay in Minutes','Arrival Delay in Minutes']
cat_df = df[cat_columns]
real_df = df[real_columns]
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_data = ohe.fit_transform(cat_df)
one_hot_data = pd.DataFrame(one_hot_data,index=cat_df.index)
y[y=='neutral or dissatisfied']=0
y[y=='satisfied']=1
data_norm = (real_df - real_df.min())/ (real_df.max() - real_df.min())
data_norm =  pd.DataFrame(data_norm,columns=data_norm.columns,dtype=float)
one_hot_data = pd.DataFrame(one_hot_data,columns=one_hot_data.columns,dtype=float)
y= pd.DataFrame(y,dtype=float)
data_norm.columns = data_norm.columns.astype(str)
one_hot_data.columns = one_hot_data.columns.astype(str)
train_real_data, test_real_data, \
train_one_hot_data, test_one_hot_data, \
y_train, y_test = train_test_split(data_norm, one_hot_data,
                                   y, train_size=0.8, random_state=85)
X_train = pd.concat([train_real_data, train_one_hot_data], axis='columns')
X_test= pd.concat([test_real_data, test_one_hot_data], axis='columns')

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(params):
    # Распаковка параметров
    voting_weight_0 = params['voting_weight_0']
    voting_weight_1 = params['voting_weight_1']
    voting_weight_2 = params['voting_weight_2']

    # Создание моделей
    clf1 = RandomForestClassifier(n_estimators=150, max_depth=22, criterion="entropy")
    clf2 = xgb.XGBClassifier(n_estimators=900, max_depth=12, learning_rate=0.026087840573904744, eval_metric='logloss')
    clf3 = CatBoostClassifier(iterations=950, depth=11, learning_rate=0.010248225733869624, loss_function='Logloss', task_type="CPU", verbose=False)

    # Создание ансамбля
    eclf = VotingClassifier(estimators=[('rf', clf1), ('xgb', clf2), ('cb', clf3)], voting='soft', weights=[voting_weight_0, voting_weight_1, voting_weight_2])
    eclf.fit(X_train, y_train)

    # Оценка точности на тестовой выборке
    y_pred = eclf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    return {'loss': -acc, 'status': STATUS_OK}

# Пространство поиска гиперпараметров
space = {
    'voting_weight_0': hp.uniform('voting_weight_0', 0, 1),
    'voting_weight_1': hp.uniform('voting_weight_1', 0, 1),
    'voting_weight_2': hp.uniform('voting_weight_2', 0, 1)
}

# Оптимизация гиперпараметров
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

# Вывод лучших найденных гиперпараметров
print('Best hyperparameters:')
print('Voting weights:', [best['voting_weight_0'], best['voting_weight_1'], best['voting_weight_2']])

Пробую ансамбль(Catboost, XGBoost, Random Forest)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier, BaggingClassifier, AdaBoostClassifier
import csv
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer

with open('public_data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

df = pd.read_csv('public_data.csv')
mean = df["Arrival Delay in Minutes"].mean()

for i in range(len(data)):
  if '' in data[i]:
    data[i][data[i].index('')] =  mean

with open('public_data.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)
df = pd.read_csv('public_data.csv',index_col='Row_id')
df=df.drop(columns=["id"])
y = df['satisfaction']

cat_columns = ['Gender','Customer Type','Type of Travel','Class']
real_columns = ['Age','Flight Distance','Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment',
                'On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness','Departure Delay in Minutes','Arrival Delay in Minutes']
cat_df = df[cat_columns]
real_df = df[real_columns]
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_data = ohe.fit_transform(cat_df)
one_hot_data = pd.DataFrame(one_hot_data,index=cat_df.index)
y[y=='neutral or dissatisfied']=0
y[y=='satisfied']=1
data_norm = (real_df - real_df.min())/ (real_df.max() - real_df.min())
data_norm =  pd.DataFrame(data_norm,columns=data_norm.columns,dtype=float)
one_hot_data = pd.DataFrame(one_hot_data,columns=one_hot_data.columns,dtype=float)
y= pd.DataFrame(y,dtype=float)
data_norm.columns = data_norm.columns.astype(str)
one_hot_data.columns = one_hot_data.columns.astype(str)
train_real_data, test_real_data, \
train_one_hot_data, test_one_hot_data, \
y_train, y_test = train_test_split(data_norm, one_hot_data,
                                   y, train_size=0.8, random_state=85)
X_train = pd.concat([train_real_data, train_one_hot_data], axis='columns')
X_test= pd.concat([test_real_data, test_one_hot_data], axis='columns')


clf1 = RandomForestClassifier(n_estimators=150, max_depth=22, criterion="entropy")
clf2 = xgb.XGBClassifier(n_estimators = 900, max_depth = 12, learning_rate = 0.026087840573904744, eval_metric='logloss')
clf3 = CatBoostClassifier(iterations = 950, depth = 11, learning_rate = 0.010248225733869624,loss_function='Logloss',task_type="CPU")
eclf = VotingClassifier([('rf', clf1), ('xgb', clf2), ('cat', clf3)], voting="soft", weights=[0.1, 0.6, 0.3])
eclf.fit(X_train, y_train)
y_pred = eclf.predict(X_test)
print(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
<ipython-input-14-c78c6298d46b>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y=='neutral or dissatisfied']=0
<ipython-input-14-c78c6298d46b>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y=='satisfied']=1
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d 

Learning rate set to 0.135064
0:	learn: 0.4483830	total: 211ms	remaining: 1m 45s
1:	learn: 0.3077547	total: 429ms	remaining: 1m 46s
2:	learn: 0.2444229	total: 640ms	remaining: 1m 45s
3:	learn: 0.1951757	total: 850ms	remaining: 1m 45s
4:	learn: 0.1694514	total: 1.05s	remaining: 1m 44s
5:	learn: 0.1507563	total: 1.25s	remaining: 1m 43s
6:	learn: 0.1395135	total: 1.48s	remaining: 1m 44s
7:	learn: 0.1290031	total: 1.68s	remaining: 1m 43s
8:	learn: 0.1206619	total: 1.88s	remaining: 1m 42s
9:	learn: 0.1153109	total: 2.09s	remaining: 1m 42s
10:	learn: 0.1092667	total: 2.3s	remaining: 1m 42s
11:	learn: 0.1056443	total: 2.52s	remaining: 1m 42s
12:	learn: 0.1026604	total: 2.74s	remaining: 1m 42s
13:	learn: 0.1003614	total: 2.95s	remaining: 1m 42s
14:	learn: 0.0986305	total: 3.15s	remaining: 1m 41s
15:	learn: 0.0969978	total: 3.35s	remaining: 1m 41s
16:	learn: 0.0951788	total: 3.58s	remaining: 1m 41s
17:	learn: 0.0934603	total: 3.77s	remaining: 1m 41s
18:	learn: 0.0915671	total: 3.99s	remaining: 

Чистый catboost

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
import csv
with open('public_data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

df = pd.read_csv('public_data.csv')
mean = df["Arrival Delay in Minutes"].mean()

for i in range(len(data)):
  if '' in data[i]:
    data[i][data[i].index('')] =  mean

with open('public_data.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)
df = pd.read_csv('public_data.csv',index_col='Row_id')
df=df.drop(columns=["id"])
y = df['satisfaction']

cat_columns = ['Gender','Customer Type','Type of Travel','Class']
real_columns = ['Age','Flight Distance','Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment',
                'On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness','Departure Delay in Minutes','Arrival Delay in Minutes']
cat_df = df[cat_columns]
real_df = df[real_columns]
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_data = ohe.fit_transform(cat_df)
one_hot_data = pd.DataFrame(one_hot_data,index=cat_df.index)
y[y=='neutral or dissatisfied']=0
y[y=='satisfied']=1
data_norm = (real_df - real_df.min())/ (real_df.max() - real_df.min())
data_norm =  pd.DataFrame(data_norm,columns=data_norm.columns,dtype=float)
one_hot_data = pd.DataFrame(one_hot_data,columns=one_hot_data.columns,dtype=float)
y= pd.DataFrame(y,dtype=float)
data_norm.columns = data_norm.columns.astype(str)
one_hot_data.columns = one_hot_data.columns.astype(str)
train_real_data, test_real_data, \
train_one_hot_data, test_one_hot_data, \
y_train, y_test = train_test_split(data_norm, one_hot_data,
                                   y, train_size=0.75)
X_train = pd.concat([train_real_data, train_one_hot_data], axis='columns')
X_test= pd.concat([test_real_data, test_one_hot_data], axis='columns')

In [ ]:
clf3 = CatBoostClassifier(iterations=500,depth = 12,loss_function='Logloss',task_type="CPU")

clf3.fit(X_train, y_train)
y_pred = clf3.predict(X_test)

print('Точность: ', accuracy_score(y_test, y_pred))